# Extract Planck instrument parameters

In [ ]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from the Planck NPIPE instrument models",
]

In [ ]:
import numpy as np
from astropy.io import fits

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from pathlib import Path

# Check if the file does not exist before downloading
file_path = Path("PLANCK_RIMO_TF_R4.00.tar.gz")
if not file_path.exists():
    !wget https://portal.nersc.gov/cfs/cmb/planck2020/misc/PLANCK_RIMO_TF_R4.00.tar.gz
    !tar xzvf PLANCK_RIMO_TF_R4.00.tar.gz

In [ ]:
RIMO = dict(
    HFI=fits.open("simulated_maps/npipe_aux/RIMO_HFI_NPIPE.fits"),
    LFI=fits.open("simulated_maps/npipe_aux/RIMO_LFI_NPIPE.fits"),
)

In [ ]:
from pathlib import Path

In [ ]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [ ]:
table = QTable(
    names=[
        "telescope",
        "band",
        "center_frequency",
        "fwhm",
        "nside",
        "bandpass_file",
        "beam_file",
    ],
    dtype=[str, str, float, float, int, str, str],
    units=[None, None, u.GHz, u.arcmin, None, None, None],
)

In [ ]:
# Add rows for HFI and LFI
for instrument in ["LFI", "HFI"]:
    for band in RIMO[instrument][
        (
            "FREQUENCY MAP PARAMETERS"
            if instrument == "HFI"
            else "FREQUENCY_MAP_PARAMETERS"
        )
    ].data:
        table.add_row(
            OrderedDict(
                telescope=instrument,
                band=band["FREQUENCY"],
                center_frequency=float(band["FREQUENCY"]) * u.GHz,
                fwhm=band["FWHM"] * u.arcmin,
                nside=2048 if instrument == "HFI" else 1024,
                bandpass_file="bandpass_" + band["FREQUENCY"] + ".tbl",
                beam_file="window_function_" + band["FREQUENCY"] + ".tbl",
            )
        )

In [ ]:
table

## Create bandpass files

In [ ]:
for band in ["30", "44", "70"]:
    file_path = Path(f"bp_corrected_{band}.dat")
    if not file_path.exists():
        !wget http://sdc.uio.no/vol/cosmoglobe-data/BeyondPlanck/releases/v2/BP_bandpass_profiles/bp_corrected_{band}.dat

In [ ]:
for band in ["030", "044", "070"]:
    corrected = np.loadtxt(f"bp_corrected_{band[1:]}.dat", unpack=True)
    plt.figure()
    plt.title(f"Band {band}")
    plt.semilogy(
        RIMO["LFI"]["BANDPASS_" + band].data["WAVENUMBER"],
        RIMO["LFI"]["BANDPASS_" + band].data["TRANSMISSION"],
        label="NPIPE",
    )
    mask = corrected[1] > 1e-6
    plt.semilogy(corrected[0][mask], corrected[1][mask], label="BeyondPlanck")

In [ ]:
from astropy import constants as const

# High-Frequency Instrument (HFI) Transmission

In HFI, the transmission $\tau$ is:

$$
m_i = K_i \frac{1 + \eta_i}{2} \epsilon_i \int d\nu (A\Omega)_\nu \tau_i(\nu) dI_\nu \quad [K_{\text{CMB}}]
$$

So, it factors out the effect of $\Omega$:

$$
\Omega \propto \frac{1}{\nu^2}
$$

Instead, in LFI it is just power:

## Receiver Bandpass

The receiver bandpass, $G(\nu)$, can be determined by performing two measurements per each frequency at different input power levels:

$$
G(\nu) = \frac{\Delta V_{\text{out}}(\nu)}{\Delta P_{\text{in}}(\nu)}.
$$

Equation (2.9)

## Implementation in PySM

Given in PySM, I use the bandpass as it is. I leave the LFI as they are, and I multiply the HFI bandpasses by $\nu^2$.  
I also checked in the color correction code, they do:

```python
trans[PosFreq] = trans[PosFreq] * (wn[PosFreq] / NUC_) ** (-2d)
```

where $\text{NUC}$ is 30, 44, 70 GHz and $\text{wn}$ is $\nu$.  

So it makes sense that the code divides the LFI bandpasses by $\nu^2$ to get to the same convention of HFI.


I also need to shift by half an interval the LFI bandpasses because instead of marking the center of an interval, they mark the left edge.

In [ ]:
skip = {"217": 2, "353": 3, "545": 9, "857": 12}
bandpass = {}
for ch in table:
    band = ch["band"]
    if ch["telescope"] == "HFI":
        freq = (
            RIMO["HFI"]["BANDPASS_F" + band].data["WAVENUMBER"] * (1 / u.cm) * const.c
        ).to(u.GHz)
        weight = RIMO["HFI"]["BANDPASS_F" + band].data["TRANSMISSION"]
        weight *= (freq / ch["center_frequency"]) ** 2
        weight /= weight.max()
        mask = np.logical_and(
            freq > 0.5 * ch["center_frequency"], freq < 1.5 * ch["center_frequency"]
        )
        mask = np.logical_and(mask, weight > 1e-2)
        freq = freq[mask]
        weight = weight[mask]
        if band in skip:
            freq = freq[:: skip[band]]
            weight = weight[:: skip[band]]
    else:
        corrected = np.loadtxt(f"bp_corrected_{band[1:]}.dat", unpack=True)
        df = corrected[0][1] - corrected[0][0]
        freq = corrected[0] + df / 2
        weight = corrected[1]
        weight = weight / weight.max()
        mask = weight > 1e-2
        weight = weight[mask]
        freq = freq[mask]
    bandpass[band] = QTable(
        names="bandpass_frequency bandpass_weight".split(),
        units=[u.GHz, None],
        data=[freq, weight],
    )
    bandpass[band].meta["comments"] = ipac_warning
    bandpass[band].write(f"bandpass_{band}.tbl", format="ascii.ipac", overwrite=True)

In [ ]:
for instrument in ["HFI", "LFI"]:
    for plot in [plt.plot, plt.semilogy]:
        plt.figure()
        plt.title(f"Planck {instrument} Bandpasses (number of points)")
        for row in table:
            if row["telescope"] == instrument:
                plot(
                    bandpass[row["band"]]["bandpass_frequency"],
                    bandpass[row["band"]]["bandpass_weight"],
                    label=f"{row['band']} ({len(bandpass[row['band']])})",
                )
        plt.xlabel("Frequency (GHz)")
        plt.ylabel("Transmission")
        plt.grid()
        plt.legend()
    plt.show()